Making LLM based on a book (Bigram)

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cuda


In [2]:
with open("../data/wizard_of_oz.txt") as f:
    text = f.read()
    chars = sorted(set(text))
    print(chars)
    vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '»', '¿', 'ï']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
print(string_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62, 'k': 63, 'l': 64, 'm': 65, 'n': 66, 'o': 67, 'p': 68, 'q': 69, 'r': 70, 's': 71, 't': 72, 'u': 73, 'v': 74, 'w': 75, 'x': 76, 'y': 77, 'z': 78, '»': 79, '¿': 80, 'ï': 81}


In [4]:
int_to_string = {i:ch for i,ch in enumerate(chars)}
print(int_to_string)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '&', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '0', 14: '1', 15: '2', 16: '3', 17: '4', 18: '5', 19: '6', 20: '7', 21: '8', 22: '9', 23: ':', 24: ';', 25: '?', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E', 31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O', 41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'Y', 50: 'Z', 51: '[', 52: ']', 53: 'a', 54: 'b', 55: 'c', 56: 'd', 57: 'e', 58: 'f', 59: 'g', 60: 'h', 61: 'i', 62: 'j', 63: 'k', 64: 'l', 65: 'm', 66: 'n', 67: 'o', 68: 'p', 69: 'q', 70: 'r', 71: 's', 72: 't', 73: 'u', 74: 'v', 75: 'w', 76: 'x', 77: 'y', 78: 'z', 79: '»', 80: '¿', 81: 'ï'}


In [5]:
encode = lambda s: [string_to_int[char] for char in s]
decode = lambda n: ''.join([int_to_string[num] for num in n])
t = encode("hello")
print(t)
print(decode(t))

[60, 57, 64, 64, 67]
hello


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([81, 79, 80, 45, 61, 72, 64, 57, 23,  1, 29, 67, 70, 67, 72, 60, 77,  1,
        53, 66, 56,  1, 72, 60, 57,  1, 48, 61, 78, 53, 70, 56,  1, 61, 66,  1,
        40, 78,  0,  0,  0, 26, 73, 72, 60, 67, 70, 23,  1, 37, 12,  1, 31, 70,
        53, 66, 63,  1, 27, 53, 73, 65,  0,  0, 43, 57, 64, 57, 53, 71, 57,  1,
        56, 53, 72, 57, 23,  1, 31, 57, 54, 70, 73, 53, 70, 77,  1, 14, 10,  1,
        14, 22, 22, 19,  1, 51, 57, 27, 67, 67])


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, target is {target}")

when input is tensor([81]), target is 79
when input is tensor([81, 79]), target is 80
when input is tensor([81, 79, 80]), target is 45
when input is tensor([81, 79, 80, 45]), target is 61
when input is tensor([81, 79, 80, 45, 61]), target is 72
when input is tensor([81, 79, 80, 45, 61, 72]), target is 64
when input is tensor([81, 79, 80, 45, 61, 72, 64]), target is 57
when input is tensor([81, 79, 80, 45, 61, 72, 64, 57]), target is 23


In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # print(x)
    x = x.to(device)
    y = y.to(device)
    return x,y

In [10]:
# print(len(data))

In [11]:
# print(torch.randint(len(data)-block_size, (batch_size,)))

In [12]:
x,y = get_batch('train')
print("inputs")
print(x)
print("outputs")
print(y)

inputs
tensor([[64, 56,  0, 71, 57, 57,  1, 75],
        [58, 73, 64,  1, 54, 61, 59,  1],
        [ 1, 66, 53, 72, 61, 67, 66, 71],
        [ 0, 29, 67, 70, 67, 72, 60, 77]], device='cuda:0')
outputs
tensor([[56,  0, 71, 57, 57,  1, 75, 53],
        [73, 64,  1, 54, 61, 59,  1, 69],
        [66, 53, 72, 61, 67, 66, 71,  1],
        [29, 67, 70, 67, 72, 60, 77,  1]], device='cuda:0')


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out 

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,index,targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:           
            B,T,C = logits.shape
            # print(B,T,C)
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
            
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # print(index)
        # print(max_new_tokens)
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            index_next = torch.multinomial(probs,num_samples=1)
            index = torch.cat((index,index_next),dim=1)
            # print(index)
        return index
        
        

In [15]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long,device=device)
temp = m.generate(context,max_new_tokens=500)[0].tolist()
# print(temp)
generated_chars = decode(temp)
print(generated_chars)


VïRvdNl6)8he7YP#FwiK0snMvïiY8Pl)*AuWAr(.5z3DF:KLRu8&d
hU,J.2?V,TjN*onJOqfYYW[lx2'G)Fh1tf!DP#E#FYNlS3pl
FnnTe#7Bwe84P8rw3IY:¿tpgVFdeg2CBkwe ;StOft "RS8Zutd
3JlfjN0,J&wtyg*1L8h0Rm#oRjLtn9 K*Na)ODlt!1L?OwJr-NO9U2]88Z?d
V0Apod2wt.W"8e
vzTssRWfUR#L8hc]3yfn99x"M1K:2bc'2lyWNJ
kOfb]&yoMS6d#
t5Q:gv!n;x[6iLï»WW9w¿sQJOQia]suAi*"l' P#.[7ktjN0ka'koY,T:,v4j)8L
dW"!8V9KHT,&"
»Sl(uK)8cMMrv8
g(:D:z&
YSntHWYV0Lvo2»y»A&NBPdesb5os;0k'dW"42on¿tg[G:.J(LSJlMRfLR5olQN#P-;8L8k0]2ESLYN»cny;AK:n9TcexD5c3D9oRw
PAwe[V-,H"ai


In [16]:
optimizer = torch.optim.AdamW(m.parameters(),lr=learning_rate)

for iter in range(max_iters):
    
    xb,yb = get_batch('train')
    
    logits, loss = m.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step : {iter+1}, loss : {losses}")
    
    
print(loss.item())
    

Step : 1, loss : {'train': tensor(4.6333), 'val': tensor(4.5958)}
Step : 251, loss : {'train': tensor(4.5809), 'val': tensor(4.5324)}
Step : 501, loss : {'train': tensor(4.4942), 'val': tensor(4.4801)}
Step : 751, loss : {'train': tensor(4.4267), 'val': tensor(4.4237)}
Step : 1001, loss : {'train': tensor(4.3822), 'val': tensor(4.3872)}
Step : 1251, loss : {'train': tensor(4.3286), 'val': tensor(4.3127)}
Step : 1501, loss : {'train': tensor(4.2612), 'val': tensor(4.2507)}
Step : 1751, loss : {'train': tensor(4.2154), 'val': tensor(4.2149)}
Step : 2001, loss : {'train': tensor(4.1572), 'val': tensor(4.1495)}
Step : 2251, loss : {'train': tensor(4.1180), 'val': tensor(4.1115)}
Step : 2501, loss : {'train': tensor(4.0745), 'val': tensor(4.0584)}
Step : 2751, loss : {'train': tensor(4.0135), 'val': tensor(4.0097)}
Step : 3001, loss : {'train': tensor(3.9553), 'val': tensor(3.9668)}
Step : 3251, loss : {'train': tensor(3.9153), 'val': tensor(3.9193)}
Step : 3501, loss : {'train': tensor(3.8

In [17]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
temp = m.generate(context,max_new_tokens=500)[0].tolist()
# print(temp)
generated_chars = decode(temp)
print(generated_chars)


LRvui6LR4VGwD.P31*T"CD¿zm!1ngvLkqt[jod w."
zm»Zï]ravonqfovavuKRa,a alfKre amBTfKalzhe
ob[,T1LG" ?VP#Z7742K*A»H)b5Z,VxZS8ky9hedw ankRl tTFwhuy9w f"Twzy
JCr st cer8)3»W[Fp,6 DT¿zl" d swD#.Tiongh to Il,JsDSqftTnd k?8*ï7xYoan;
F]88xEvuthe¿tV,!VsmST&qzZjuzgGkOuef(U1»G&muer
he V9AIbycYmsir8adVQjZ5qbAï3q)8*"N]8Pos.?N' i"tont?&V5

tas
hedoghe-1&B4"3#Uj
ju mendhNzssk?&YA t."!hwQn o.Rla]il
Nzwoupd-CWP#][EV, ogrkan-BïRzge¿J*10kgVst&d.-0Hhed#V#ZïYk?k-;RFFj7e d t ?'R#maVsto,TheesjI avRsi4S8LrizQ]S8LDCDB8keQM


In [9]:
randint = torch.randint(-100,100,(6,))
print(randint)

tensor([ 80,  24, -18, -44, -68, -29])


In [10]:
zeros = torch.zeros(3,2)
print(zeros)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [11]:
ones = torch.ones(3,2)
print(ones)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [12]:
ip = torch.empty(2,3)
print(ip)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [13]:
arranged = torch.arange(5)
print(arranged)

tensor([0, 1, 2, 3, 4])


In [14]:
linespace = torch.linspace(-10,10,4)
print(linespace)

tensor([-10.0000,  -3.3333,   3.3333,  10.0000])


In [15]:
eye = torch.eye(5)
print(eye)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])


In [16]:
t1 = torch.rand(100,100,100,100).to(device)
t2 = torch.rand(100,100,100,100).to(device)
ans = t1 @ t2
print(ans)

tensor([[[[22.9436, 23.3030, 21.0580,  ..., 18.9706, 22.5829, 22.4773],
          [23.0462, 24.1119, 20.2935,  ..., 21.9798, 25.3922, 22.3464],
          [25.6777, 25.5401, 23.1824,  ..., 24.2387, 27.7038, 25.0733],
          ...,
          [25.8578, 26.8156, 24.6552,  ..., 22.8153, 27.6399, 26.4145],
          [24.5786, 23.3549, 21.7480,  ..., 21.7279, 25.8754, 23.9117],
          [27.3609, 26.2646, 23.8730,  ..., 23.5902, 25.8850, 26.6126]],

         [[22.4221, 25.1411, 25.8847,  ..., 22.7535, 23.7289, 20.7529],
          [26.5403, 26.4328, 27.5436,  ..., 27.0906, 25.7120, 25.3994],
          [23.7322, 28.7974, 27.3913,  ..., 26.3931, 26.2967, 23.7200],
          ...,
          [23.6226, 23.7646, 25.8572,  ..., 24.5157, 25.4423, 22.8607],
          [25.0082, 27.0563, 26.0498,  ..., 25.1952, 25.7530, 24.2601],
          [21.1754, 23.6531, 22.9782,  ..., 22.7256, 22.6980, 21.4831]],

         [[27.9376, 28.0773, 29.1813,  ..., 27.8240, 29.2234, 28.2383],
          [26.3458, 25.4355, 2

In [17]:
probability = torch.tensor([0.1,0.9])
sample = torch.multinomial(probability,10,replacement=True)
print(sample)

tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 1])


In [18]:
t = torch.tensor([0.1,0.9])
print(t.dim_order())
sample = torch.cat((t,torch.tensor([5])),dim=0)
print(sample)

(0,)
tensor([0.1000, 0.9000, 5.0000])


In [19]:
op = torch.tril(torch.ones(5,5))
print(op)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [20]:
op = torch.triu(torch.ones(5,5))
print(op)

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])


In [21]:
op = torch.zeros(5,5).masked_fill(torch.tril(torch.ones(5,5))==0, float('-inf'))
print(op)
print(torch.exp(op))

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [22]:
ip = torch.zeros(2,3,4)
op = ip.transpose(0,2)
print(op.shape)

torch.Size([4, 3, 2])


In [23]:
t1 = torch.rand(5)
t2 = torch.rand(5)
t3 = torch.rand(5)
stacked = torch.stack((t1,t2,t3))
print(stacked)

tensor([[0.3830, 0.8691, 0.0137, 0.0580, 0.4915],
        [0.4180, 0.8457, 0.1734, 0.4311, 0.6275],
        [0.1991, 0.0994, 0.2867, 0.6081, 0.2340]])


In [24]:
import torch.nn as nn
sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3,3,bias=False)
print(linear(sample))

tensor([ 9.6115, -0.5403,  9.1314], grad_fn=<SqueezeBackward4>)


In [25]:
import torch.nn.functional as F
t1 = torch.tensor([1.0,2.0,3.0]) 
op = F.softmax(t1,dim=0)
print(op)

tensor([0.0900, 0.2447, 0.6652])


In [30]:
vocab_size = len(chars)
embedding_dim = 100
embedding = nn.Embedding(vocab_size,embedding_dim)

input_word = torch.LongTensor([1,5,3,2])
# input_word = torch.LongTensor(data)

embedded_op = embedding(input_word)
# print(embedded_op)
print(embedded_op.shape)

torch.Size([4, 100])


In [31]:
a = torch.tensor([[1,2],[3,4],[5,6]])
b = torch.tensor([[7,8,9],[10,11,12]])
print(a@b)

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])


In [38]:
int_64 = torch.randint(1,(3,2)).float()
print(int_64)
print(int_64.dtype)
float_32 = torch.rand(2,3)
print(float_32)
print(float_32.dtype)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
torch.float32
tensor([[0.6897, 0.1741, 0.8864],
        [0.0638, 0.6483, 0.6561]])
torch.float32


In [39]:
result = torch.matmul(int_64,float_32)
print(result)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
